Disqus
======

- https://disqus.com/api/docs/requests/

Constructing Requests

Requests are made of three components:

    API version
    Resource path
    Output type

To construct a proper request, you will need to format the URI as follows:

https://disqus.com/api/{version}/{resource}.{output_type}

An example request, to fetch a list of posts on a forum might be:

https://disqus.com/api/3.0/forums/listPosts.json?forum=disqus

Authentication
--------------

- Secret Key: a2SOi6pc5pJSbHsMTP2EekxnbTtSIygYq1BU8aQBbnAyfHwhuhaUZQWJ5CMHVfBr

- Public Key: N6T89OylJj7sP3NGjkMNtqQkbdsZT6LjXpJf4LGCbFcVros2RhaesPsZ7iPiUmgg

- Domains: nedlaw.eu, disqus.com 

Links
-----

- https://github.com/disqus/DISQUS-API-Recipes/ - git cookbook
- https://help.disqus.com/customer/en/portal/articles/1131785-available-public-api-data - available public data

Disqus module problem

- https://github.com/mishbahr/djangocms-disqus/issues/1
- https://github.com/disqus/disqus-python

available user details
- https://disqus.com/api/docs/users/

exameple string
- https://disqus.com/api/3.0/forums/listPosts.json?forum=disqus
- https://disqus.com/api/3.0/trends/listThreads.json?api_key=API_PUBLIC_KEY_HERE&callback=foo"

In [16]:
from disqusapi import DisqusAPI

secret_key = 'a2SOi6pc5pJSbHsMTP2EekxnbTtSIygYq1BU8aQBbnAyfHwhuhaUZQWJ5CMHVfBr'
public_key = 'N6T89OylJj7sP3NGjkMNtqQkbdsZT6LjXpJf4LGCbFcVros2RhaesPsZ7iPiUmgg'
access = '5b856d561154418cb6c4619f9816f3d0'

disqus = DisqusAPI(secret_key, public_key)

In [22]:
import base64
import hashlib
import hmac
import simplejson
import time
 
DISQUS_SECRET_KEY = 'a2SOi6pc5pJSbHsMTP2EekxnbTtSIygYq1BU8aQBbnAyfHwhuhaUZQWJ5CMHVfBr'
DISQUS_PUBLIC_KEY = 'N6T89OylJj7sP3NGjkMNtqQkbdsZT6LjXpJf4LGCbFcVros2RhaesPsZ7iPiUmgg'

userName = 'Maquirer'
email = 'markus.antero.walden@gmail.com'
ids = 'maguire'

In [35]:
def get_disqus_sso(user):
    # create a JSON packet of our data attributes
    data = simplejson.dumps({
        'id': user['id'],
        'username': user['username'],
        'email': user['email'],
    })
    print (data)
    # encode the data to base64
    message = base64.b64encode(data)
    # generate a timestamp for signing the message
    timestamp = int(time.time())
    # generate our hmac signature
    sig = hmac.HMAC(DISQUS_SECRET_KEY, '%s %s' % (message, timestamp), hashlib.sha1).hexdigest()
 
    # return a script tag to insert the sso message
    return """<script type="text/javascript">
    var disqus_config = function() {
        this.page.remote_auth_s3 = "%(message)s %(sig)s %(timestamp)s";
        this.page.api_key = "%(pub_key)s";
    }
    </script>""" % dict(
        message=message,
        timestamp=timestamp,
        sig=sig,
        pub_key=DISQUS_PUBLIC_KEY,
    )

In [36]:
user = {'id': ids, 'username': userName , 'email': email}
t = get_disqus_sso(user)

print (t)

{"id": "maguire", "email": "markus.antero.walden@gmail.com", "username": "Maquirer"}


TypeError: a bytes-like object is required, not 'str'


- https://github.com/disqus/DISQUS-API-Recipes/blob/master/oauth/python/oauth_example.py


In [ ]:
from flask import session
from flask import redirect
from flask import url_for
from flask import escape
from flask import request

import functools
import simplejson
import urllib

from disqusapi import InvalidAccessToken
from disqusapi import DisqusAPI

In [11]:
class User(object):
    """User object based on disqus auth token"""
    def __init__(self, username, user_id, access_token, expires_in, token_type, refresh_token):
        super(User, self).__init__()
        self.username = username
        self.user_id = user_id
        self.access_token = access_token
        self.expires_in = expires_in
        self.token_type = token_type
        self.refresh_token = refresh_token

    def __repr__(self):
        return "<{username}:{access_token}>".format(**self.__dict__)


def current_user():
    cu = None
    if 'auth' in session:
        auth = session['auth']
        cu = User(**auth)
    return cu

In [26]:
from flask import Flask
app = Flask(__name__)

app.secret_key = "this is my salt"
DISQUS_SECRET_KEY = 'a2SOi6pc5pJSbHsMTP2EekxnbTtSIygYq1BU8aQBbnAyfHwhuhaUZQWJ5CMHVfBr'
DISQUS_PUBLIC_KEY = 'N6T89OylJj7sP3NGjkMNtqQkbdsZT6LjXpJf4LGCbFcVros2RhaesPsZ7iPiUmgg'

userName = 'Maquirer'
email = 'markus.antero.walden@gmail.com'
ids = 'maguire'

@app.route("/")
def hello(userName):
    cu = current_user()
    if cu:
        return 'Logged in as %s' % escape(session[userName])
    else:
        return redirect('/oauth/authorize/')


@app.route("/bye")
def goodbye():
    return "Goodbye World!"


@app.route("/foo/<int:bar>/<baz>")
def foobarbaz(bar, baz):
    if bar == 1:
        return "yay it's a one!"
    elif bar == 2:
        return "two"
    else:
        return baz

In [27]:
class Logout(Exception):
    pass

def api_call(func, **kwargs):
    access = '5b856d561154418cb6c4619f9816f3d0'
    try:
        if 'auth' in session:
            result = func(access_token=session['auth'][access], **kwargs)
        else:
            result = func(**kwargs)
    except InvalidAccessToken:
        raise Logout
    return result


@app.errorhandler(Logout)
def logout_handler(error):
    try:
        del session['auth']
    except KeyError:
        pass
    return redirect(url_for('oauth_authorize'))


def login_required(func):
    @functools.wraps(func)
    def wrapped(*args, **kwargs):
        if 'auth' not in session:
            return redirect(url_for('oauth_authorize'))
        return func(*args, **kwargs)
    return wrapped


@app.route('/oauth/authorize/')
def oauth_authorize():
    return redirect('https://disqus.com/api/oauth/2.0/authorize/?%s' % (urllib.urlencode({
        'client_id': disqus.public_key,
        'scope': 'read,write',
        'response_type': 'code',
        'redirect_uri': url_for('oauth_callback', _external=True),
    }),))


@app.route('/oauth/callback/')
def oauth_callback():
    code = request.args.get('code')
    error = request.args.get('error')
    access = '5b856d561154418cb6c4619f9816f3d0'
    
    if error or not code:
        # TODO: show error
        return redirect('/')

    req = urllib2.Request('https://disqus.com/api/oauth/2.0/' + access + '/', urllib.urlencode({
        'grant_type': 'authorization_code',
        'client_id': disqus.public_key,
        'client_secret': disqus.secret_key,
        'redirect_uri': url_for('oauth_callback', _external=True),
        'code': code,
    }))
    
    resp = urllib2.urlopen(req).read()

    data = simplejson.loads(resp)

    session['auth'] = data
    session['username'] = data['username']
    session.permanent = True

    return redirect('/')



In [28]:
if __name__ == "__main__":
    app.config["DEBUG"] = True
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

C:\python3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
